# Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torch
import torch.nn as nn
import torch.optim as optim

# Dataset

In [4]:
# Caricare i dataset
train_data = pd.read_csv('DatasetClassification/TrainDataset.csv')
test_data = pd.read_csv('DatasetClassification/TestDataset.csv')

# Separare caratteristiche (X) e label (y)
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']
X_test = test_data.drop(columns=['label'])
y_test = test_data['label']

# Suddividere il training set in TR e VL
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)  # 20% per la validazione

# Convertire in tensori per PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Creare DataLoader per la rete neurale
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=100, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=100, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=100, shuffle=False)


551662
35320
165499
49650


# Tools

In [56]:
import torch
from sklearn.metrics import accuracy_score as a_s

accuracy_score = lambda x, y: a_s(x.detach().numpy(), y.detach().numpy())

def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, epochs=10, stop_criteria=lambda epoch: False):
    
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []

    epoch = 0

    while (epoch < epochs and not stop_criteria(epoch)):
        
        # Modalità di allenamento
        model.train()
        
        batch_train_loss = []
        batch_train_acc = []
        
        for X_batch, y_batch in train_loader:
            
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            
            print(accuracy_score(y_batch, output))
            
            batch_train_loss.append(loss.item())
            batch_train_acc.append(accuracy_score(y_batch.detatch().numpy(), output))
            
        train_losses.append(np.mean(batch_train_loss))
        train_acc.append(np.mean(batch_train_acc))

        # Modalità di validazione
        model.eval()
        
        batch_val_loss = []
        batch_val_acc = []
        
        with torch.no_grad():
            
            for X_batch, y_batch in val_loader:
                
                output = model(X_batch)
                loss = criterion(output, y_batch)
                
                batch_val_loss.append(loss.item())
                batch_val_acc.append(accuracy_score(y_batch, output))

        val_losses.append(np.mean(batch_val_loss))
        val_acc.append(np.mean(batch_val_acc))

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_losses[-1]}")
        print(f"Train Accuracy: {train_acc[-1]}")
        print(f"Val Loss: {val_losses[-1]}")
        print(f"Val Accuracy: {val_acc[-1]}")
        print("")

        epoch += 1

    # Valutazione sul set di test
    model.eval()
    
    batch_test_loss = []
    batch_test_acc = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            
            output = model(X_batch)
            loss = criterion(output, y_batch)
                
            batch_test_loss.append(loss.item())
            batch_test_acc.append(accuracy_score(y_batch, output))
            
    print(f"Final Test Accuracy: {np.mean(batch_test_acc)}")
    print(f"Final Test Loss: {np.mean(batch_test_loss)}")    

# Model

## Linear classification

In [57]:
class LinearClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LinearClassifier, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        return self.linear(x)

# Inizializzare modello, loss e ottimizzatore
model = LinearClassifier(input_size=X_train.shape[1], num_classes=len(np.unique(y_train)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, 100)

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets